In [ ]:
import pandas as pd
import bs4
import selenium
import requests
import re
from selenium import webdriver

In [8]:
browser = webdriver.Chrome('chromedriver.exe')

browser.get('https://www.zomato.com/bangalore/great-food-no-bull')

In [9]:
res =browser.find_elements_by_css_selector('div.col-s-8.col-l-1by3')
print(len(res))

142


In [13]:
from selenium.common.exceptions import NoSuchElementException
def get_info(review, selector , attribute):
    try:    
        tag = review.find_element_by_css_selector(selector)
    
        tag = tag.get_attribute(attribute)
    except:
        tag=None
    return tag

df_rest=pd.DataFrame(columns={'restaurant_name','restaurant_link'})

for rest in res:  
    restaurant_name = get_info(rest,'div.res_title.zblack.bold.nowrap','innerHTML')
    if restaurant_name != None:
        restaurant_name = restaurant_name.strip()
    restaurant_link = get_info(rest,'a.relative.top-res-box-bg.pl10.ptop0','href')
    curr_restaurant={
                 'restaurant_name':restaurant_name,
                 'restaurant_link':restaurant_link
             }
    df_rest= df_rest.append(curr_restaurant,ignore_index=True)
    
df_rest.head()

,restaurant_link,restaurant_name
0,https://www.zomato.com/bangalore/roots-koraman...,Roots
1,https://www.zomato.com/bangalore/vapour-brewpu...,Vapour Brewpub and Diner
2,https://www.zomato.com/bangalore/caperberry-la...,Caperberry
3,https://www.zomato.com/bangalore/echoes-korama...,ECHOES Koramangala
4,https://www.zomato.com/bangalore/the-reservoir...,The Reservoire


In [16]:
df_reviews=pd.DataFrame(columns={'rating','user_name','user_id','rest_name'})
for link in df_rest['restaurant_link']:
    browser.get(link )
    while True:
        more_sel = 'div.load-more'
        try:
            load_more_button = browser.find_element_by_css_selector(more_sel)
            load_more_button.click()

            reviews = browser.find_elements_by_css_selector('div.ui.segment.clearfix.brtop')
        except Exception:
            break
    def get_info(review, selector , attribute):
        try:    
            tag = review.find_element_by_css_selector(selector)

            tag = tag.get_attribute(attribute)
        except:
            tag=None
        return tag
        
    for review in reviews:  
            rating=get_info(review,'div.ttupper','aria-label')
            #y=re.findall('[0-9.]+',rating)
            #rating = float(y[0])
            name_selec = 'a[data-entity_id]'
            user_id =get_info(review,name_selec,'data-entity_id')  
            user_name = get_info(review,name_selec,'innerHTML') 
            if user_name !=None:
                user_name = user_name.strip()
            rest_name = get_info(browser,'a.ui.large.header','title') 
            curr_review={
                         'user_name':user_name,
                         'rating':rating,
                         'user_id':user_id,
                         'rest_name':rest_name
                     }
            df_reviews= df_reviews.append(curr_review,ignore_index=True)
df_reviews.head()  

,user_id,rest_name,user_name,rating
0,15880153,Roots Bangalore,Madhurima Deka,Rated 3.0
1,812105,Roots Bangalore,Joyjit,Rated 4.0
2,1384839,Roots Bangalore,Vivek Sinha,Rated 5.0
3,1187917,Roots Bangalore,Barna Paul,Rated 5.0
4,35233361,Roots Bangalore,Akshata Sensharma,Rated 5.0


In [17]:
df_reviews.shape[0]

11213

In [18]:
df_reviews.to_csv('zomato_scrap.csv',index=False)

In [19]:
import pandas as pd

In [21]:
zomato = pd.read_csv('C:/Users/Hitman/zomato_scrap.csv')

In [22]:
zomato.head()

,user_id,rest_name,user_name,rating
0,15880153.0,Roots Bangalore,Madhurima Deka,Rated 3.0
1,812105.0,Roots Bangalore,Joyjit,Rated 4.0
2,1384839.0,Roots Bangalore,Vivek Sinha,Rated 5.0
3,1187917.0,Roots Bangalore,Barna Paul,Rated 5.0
4,35233361.0,Roots Bangalore,Akshata Sensharma,Rated 5.0


In [23]:
zomato=zomato.dropna(axis=0)

In [24]:
zomato.isnull().sum()

user_id      0
rest_name    0
user_name    0
rating       0
dtype: int64

In [25]:
import re
list=[]
for i in zomato.rating.items():
    value = re.findall('[0-9.]+',i[1])
    list.append(value[0])

In [27]:
zomato.rating=list
zomato.rating = zomato.rating.astype(float)
zomato.user_id = zomato.user_id.astype(int)
zomato.user_id = 'User-' + zomato.user_id.astype(str)
zomato.head()

,user_id,rest_name,user_name,rating
0,User-15880153,Roots Bangalore,Madhurima Deka,3.0
1,User-812105,Roots Bangalore,Joyjit,4.0
2,User-1384839,Roots Bangalore,Vivek Sinha,5.0
3,User-1187917,Roots Bangalore,Barna Paul,5.0
4,User-35233361,Roots Bangalore,Akshata Sensharma,5.0


In [39]:
user_recommend = zomato.pivot_table(index='user_id',
                                      columns='rest_name',
                                      values='rating')

In [40]:
from sklearn.metrics.pairwise import cosine_similarity
zom_user = user_recommend.apply(lambda v:v-v.mean(),axis=1).fillna(0)

In [41]:
sim_matrix = pd.DataFrame(cosine_similarity(zom_user),index=zom_user.index,columns=zom_user.index)

In [43]:
neighs = sim_matrix['User-15880153'].drop('User-1187917').sort_values(ascending=False).head(7).index
neighs

Index(['User-15880153', 'User-681081', 'User-33958274', 'User-1972562',
       'User-38200606', 'User-2685180', 'User-50917700'],
      dtype='object', name='user_id')

In [48]:
preds = pd.DataFrame(columns=['restaurant','predicted_rating'])
for rest_name,rating in user_item_matrix.loc['User-1187917'].items():
    if pd.isnull(rating):
        rest_ratings =user_item_matrix.loc[neighs,rest_name]
        avg_rest_rating =rest_ratings.mean()
        preds =preds.append({'restaurant':rest_name,'predicted_rating':avg_rest_rating},ignore_index=True)
preds.sort_values('predicted_rating',ascending=False).head(3)

,restaurant,predicted_rating
77,Smoor Bangalore,4.714286
29,Dhaba - Estd 1986 Delhi Bangalore,4.000000
86,The Asian Curry House Bangalore,4.000000


In [49]:
def ubcf(userid):
    neighs_userid=sim_matrix[userid].drop(userid).sort_values(ascending=False).head(7).index
    preds =pd.DataFrame(columns=['rest_name','predicted_rating'])
    for rest_name,rating in user_item_matrix.loc[userid].items():
        if pd.isnull(rating):
            rest_ratings =user_item_matrix.loc[neighs_userid,rest_name]
            avg_rest_rating =rest_ratings.mean()
            preds =preds.append({'rest_name':rest_name,'predicted_rating':avg_rest_rating},ignore_index=True)

    movies_recom =preds.sort_values('predicted_rating',ascending=False).head(3)['rest_name']
    return movies_recom

In [50]:
ubcf('User-1384839')

9         Biergarten Bangalore
41    Hard Rock Cafe Bangalore
54            Maziga Bangalore
Name: rest_name, dtype: object